# ABOUT:
- we further fine tune a indonesian question answering model to predict answer spans
- given the address "toko dita, kertosono" the a question answering model should predict "toko dita" answer span

In [1]:
model_checkpoint = "cahya/bert-base-indonesian-tydiqa"
batch_size = 15

## load_from_disk

In [2]:
from datasets import load_from_disk

In [3]:
QA_dataset = load_from_disk(r"C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\datasets\shopee QA dataset")

In [4]:
from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML
def show_random_elements(dataset, num_examples = 10):
    assert num_examples <= len(dataset), "Can't pick more elements than there are in the dataset."
    picks = []
    for _ in range(num_examples):
        pick = random.randint(0, len(dataset)-1)
        while pick in picks:
            pick = random.randint(0, len(dataset)-1)
        picks.append(pick)
    
    df = pd.DataFrame(dataset[picks])
    for column, typ in dataset.features.items():
        if isinstance(typ, ClassLabel):
            df[column] = df[column].transform(lambda i: typ.names[i])
        elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
            df[column] = df[column].transform(lambda x: [typ.feature.names[i] for i in x])
    display(HTML(df.to_html()))

In [5]:
show_random_elements(QA_dataset['train'],5)

,attention_mask,end_positions,input_ids,start_positions,token_type_ids
0,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",8,"[3, 6186, 1769, 15324, 32, 1, 6318, 17, 10608, 8366, 17851, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]",6,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
1,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",7,"[3, 6186, 1769, 15324, 32, 1, 5565, 6889, 2845, 7973, 16396, 23718, 23771, 2070, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]",6,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
2,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",6,"[3, 6186, 1769, 15324, 32, 1, 1682, 15, 2845, 7006, 2472, 6496, 12044, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]",6,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
3,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",9,"[3, 3397, 6834, 1538, 9118, 32, 1, 17115, 4997, 1007, 15, 20475, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]",7,"[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"
4,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]",10,"[3, 6186, 1769, 15324, 32, 1, 3589, 11460, 1944, 2102, 4125, 3674, 8148, 4462, 19150, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]",6,"[0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]"


In [6]:
# from datasets import DatasetDict
# smol_QA_dataset = DatasetDict()
# smol_QA_dataset['train'] = QA_dataset['train'].select(range(55000,65000))
# smol_QA_dataset['test'] = QA_dataset['test'].select(range(500))
# smol_QA_dataset

## AutoModelForQuestionAnswering

In [7]:
from transformers import AutoModelForQuestionAnswering, TrainingArguments, Trainer
model = AutoModelForQuestionAnswering.from_pretrained(model_checkpoint)

In [8]:
model.to("cuda")

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(32000, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_

## TrainingArguments and Trainer

In [9]:
args = TrainingArguments(
    f"test-squad",
    evaluation_strategy = "no",          # evaluation_strategy = "epoch" means to evaluate at the end of each epoch
    # eval_steps = 100,                       # evaluate every n number of STEPS - steps = num samples/batch size
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=1,
    weight_decay=0.01,
    save_strategy = "no"
)

In [10]:
from transformers import default_data_collator
data_collator = default_data_collator

In [11]:
trainer = Trainer(
    model,
    args,
    train_dataset=QA_dataset["train"],
    # eval_dataset=smol_QA_dataset["test"],
    data_collator=data_collator,
    # tokenizer=tokenizer,
)

In [12]:
# import mlflow
# mlflow.end_run()    

In [13]:
trainer.train()

***** Running training *****
  Num examples = 273429
  Num Epochs = 1
  Instantaneous batch size per device = 15
  Total train batch size (w. parallel, distributed & accumulation) = 15
  Gradient Accumulation steps = 1
  Total optimization steps = 18229


Step,Training Loss
500,0.581400
1000,0.375700
1500,0.326900
2000,0.293000
2500,0.287900
3000,0.272600
3500,0.260200
4000,0.250400
4500,0.245700
5000,0.231500




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18229, training_loss=0.2130353980870325, metrics={'train_runtime': 4608.2801, 'train_samples_per_second': 59.334, 'train_steps_per_second': 3.956, 'total_flos': 9025477591923000.0, 'train_loss': 0.2130353980870325, 'epoch': 1.0})

## export

In [14]:
model.save_pretrained(r"C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\models\address extraction QA model")

Configuration saved in C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\models\address extraction QA model\config.json
Model weights saved in C:\Users\tanch\Documents\Coding Competitions\Shopee\Shopee Address Elements Extraction (local)\models\address extraction QA model\pytorch_model.bin


## sanity check
- the question answering model is surprising accurate compared to previous text classification models
    - 93% of spans were correct

In [15]:
output = trainer.predict(QA_dataset["test"])

***** Running Prediction *****
  Num examples = 14392
  Batch size = 15


In [16]:
def get_exact_match(output):
    start_position_pred = np.fromiter(map(np.argmax,output.predictions[0]),dtype=np.int)
    end_position_pred = np.fromiter(map(np.argmax,output.predictions[1]),dtype=np.int)
    start_position_true = output.label_ids[0]
    end_position_true = output.label_ids[1]    
    matched = np.logical_and(start_position_pred==start_position_true, end_position_pred==end_position_true)
    return sum(matched)/len(matched)                   

In [17]:
get_exact_match(output)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

0.9321150639244025